In [ ]:
from gensim import corpora, models, similarities
import json
from nltk.tokenize import word_tokenize
from gensim.similarities import MatrixSimilarity

Importando arquivos com o map

In [ ]:
try:
   with open('map_docs.json', 'r', encoding='utf-8') as file:
       map_docs = json.load(file)
   with open('map_queries.json', 'r', encoding='utf-8') as file:
       map_queries = json.load(file)
except FileNotFoundError as e:
    print(f"Erro ao abrir o arquivo: {e}")

Criando modelo TF-IDF

In [ ]:
# Listas paralelas para manter mapeamento entre índice e doc_id
doc_ids = []
texts = []

for doc_id, texto in map_docs.items():
    tokens = word_tokenize(texto.lower())
    texts.append(tokens)
    doc_ids.append(doc_id)  # manter a ordem original

In [ ]:
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
tfidf_model = models.TfidfModel(bow_corpus)

In [ ]:
def buscar_tfidf(query, top_n=10):
    query_tokens = word_tokenize(query.lower())
    query_bow = dictionary.doc2bow(query_tokens)
    query_tfidf = tfidf_model[query_bow]
    sims = index[query_tfidf]
    ranked_indices = sims.argsort()[::-1][:top_n]
    return [(doc_ids[i], sims[i]) for i in ranked_indices]

In [ ]:
query = "lung disease nitric oxide"
resultados = buscar_tfidf(query)

for doc_id, score in resultados:
    print(f"DocID: {doc_id} | Score: {score:.4f}")

Função da consulta

In [ ]:
def responder_consulta(consulta, dicionario, modelo_tfidf, index, perguntas, categorias, top_k=5):
    consulta_tokens = preprocessar(consulta)
    consulta_bow = dicionario.doc2bow(consulta_tokens)
    consulta_tfidf = modelo_tfidf[consulta_bow]
    
    sims = index[consulta_tfidf]
    sims = list(enumerate(sims))
    sims = sorted(sims, key=lambda x: -x[1])

    print(f"\nConsulta: {consulta}\n")
    print("Top resultados:")
    for idx, score in sims[:top_k]:
        print(f"[{score:.2f}] {perguntas[idx]} (Classe: {categorias[idx]})")

Testes de consulta

In [ ]:
responder_consulta("Who leads Brazil?", dicionario, modelo_tfidf, index, perguntas, categorias)